### **PLN (processamento de lingaugem natural) aplicado**
### **PIPELINE**
![alt text](img/_pipeline.png)  

**Sentença** - frase em si  
**Tokenização** - separação das palavras do texto, buscando o contexto da frase  
**(Pos) Tagging** - verifica qual o tipo da palavra verbo, pronome etc.  
**Lemmantizantion** - fatia as palavras por exemplo, 'andando' - and - andei - andarei  
**Stopwords** - palavras que podem ser removidas, sem influência no modelo
**Dependency parsing** - identifica as relações de dependência entre as palavra  
**Nouns Phrases** - (Frases Nominais) são grupos de palavras que funcionam juntas como um único substantivo na frase  
**Name Identify** recognition (NER) -  (Reconhecimento de Entidades Nomeadas) é uma técnica usada para identificar e classificar entidades nomeadas em um texto, como nomes de pessoas, locais, organizações, datas, números, entre outros.

#### **Bibliotecas utilizadas** ambas para Processamento de Linguagem Natural ( PLN ) exceto pandas, algumas apenas citadas
  **NLTK**  
  **Spacy**   
  **Open NLP**  
  **Stanza**  
  **Pandas**  
  **Azure AI Text Analysis**  
  **Azure AI Translator**
  **IBM Watson**

In [1]:
import pandas as pd # data
import string # processing
import re # processing
import nltk # processing
from nltk.corpus import stopwords # processing
from nltk.tokenize import RegexpTokenizer # processing
import unidecode # processing
import requests, uuid, json # data translator - Azure AI

# Sentiment Analysis
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

# Emotion Analysis
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions
# Se necessário 👇🏽
# pip uninstall urllib3
# pip install urllib3==1.26.8

import os
from dotenv import load_dotenv

load_dotenv()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rapha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Env's

In [2]:
azure_ai_translator_key = os.getenv("AZURE_AI_TRANSLATOR_KEY")
azure_ai_sentiment_key = os.getenv("AZURE_AI_SENTIMENT_KEY")
ibm_watson_emotion_key = os.getenv("IBM_WATSON_EMOTION_KEY")

#### Funções de tratamento de texto

In [3]:
def TextToLower(text):
    return text.lower()


def RemoveBlank(text):
    return text.strip()


def RemovePunctuation(text):
    punct = string.punctuation
    punct = punct.replace("'","")
    return text.translate(str.maketrans("","",punct))


def RemoveEmoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # simbolos
        u"\U0001F680-\U0001F6FF" # transporte
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0" 
        u"\U000024C2-\U0001F251"
                    "]+", flags = re.UNICODE)
    
    return emoji_pattern.sub(r'', text) 


def RemoveHttp(text):
    http = "https?://\S+|www\.\S+"
    pattern = r"({})".format(http)
    return re.sub(pattern, "", text)


regexp = RegexpTokenizer("[\w']+")
def RemoveStopwords(text, stopwords):
    return " ".join(word for word in regexp.tokenize(text) if word.lower() not in stopwords)



#### Tradutor de texto - Azure AI Translator

In [4]:
# Chave e endpoint
key = azure_ai_translator_key
endpoint = "https://api.cognitive.microsofttranslator.com"
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'pt',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}


def TranslateText(text):
    body = [{
        'text': text
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()

    return RemovePunctuation(json.dumps(response[0]["translations"][0]["text"], sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

print(TranslateText('Testando'))

Testing


#### Análise de sentimento - Azure AI Text Analysis

In [5]:
# Essa função funciona melhor com o texto em português, mas nada impede de fazer em inglês e ver os resultados
def SentimentAnalysis(text):
    endpoint = "https://sentiment-analysis-ds.cognitiveservices.azure.com/"
    key = azure_ai_sentiment_key

    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    result = text_analytics_client.analyze_sentiment([text], show_opinion_mining=True)
    
    sentiment = result[0]['sentiment']
   
    return sentiment 

print(SentimentAnalysis("Teste"))

neutral


#### Análise de emoção - IBM Watson

In [6]:
# config da chave API
authenticator = IAMAuthenticator(ibm_watson_emotion_key)

# Cria uma instância do serviço NLU e define a URL
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)
natural_language_understanding.set_service_url('https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/186a3145-a256-4653-945a-cb77e4542307')

In [7]:
# Função
def EmotionAnalysis(text):
    try:
        # Solicitação de análise de emoção
        response = natural_language_understanding.analyze(
            text=text,
            features=Features(emotion=EmotionOptions())
        ).get_result()

        # Extrair emoções
        emocoes = response['emotion']['document']['emotion']
        return max(emocoes, key=emocoes.get)
    except:
        return "insuficiente"

print(EmotionAnalysis("I'm happy today"))

joy


#### Leitura e tratamento dos dados
Há outros datasets com textos disponiveis em 'data'

In [8]:
ds = pd.read_csv('data/ptbr-order_reviews.csv')
df = pd.DataFrame(ds.sample(200))
df = df[["score","title","opinion"]].dropna()
# df = df[["order_id","score","title","opinion"]].dropna()
display(df)

,score,title,opinion
53289,5,Excelente fornecedor!,Entregue antes do prazo!
68254,5,Sempre perfeita,O produto é de excelente qualidade. Entrega an...
50553,1,Atraso na entrega,Comprei achando que iria receber em três dias ...
84578,4,Bom,"Produto bom, pelo preço."
89538,2,Regular,Comprei 2 produtos chegou apenas 1. Aguardo re...
46800,5,super recomendado,muito bom
31170,5,Bom demais!,A entrega do produto veio antes do esperado. A...
57893,5,Compra do produto Rebotec,Estou satisfeito com a entrega do produto. Par...
16727,5,Tudo ok,Otimo. Top de linha
44782,1,cancelamento compra,Foi cancelado a compra antes da emissão de not...


#### Tratamento e tradução do texto
Remoção de pontuações, emoji, link http, brancos, entre outros..

In [9]:
stopwords_pt = set(stopwords.words('english'))

def TextNormalizer(text):
    text = unidecode.unidecode(text)
    text = re.sub('\n', '', text)
    text = RemoveHttp(text)
    text = RemoveEmoji(text)
    text = RemovePunctuation(text)
    text = RemoveStopwords(text, stopwords_pt)
    text = RemoveBlank(text)   
    text = TextToLower(text)
    text = TranslateText(text)
    
    return text

In [10]:
# Normalização e tradução do texto
# Faça teste tanto do Azure Ai quanto do Ibm Watson com texto em inglês e português, ele varia o retorno a depender do idioma
df['opinion_en'] = df['opinion'].apply(TextNormalizer)
df['title_en'] = df['title'].apply(TextNormalizer)

#### Aplicação das funções de sentimento e emoção nos textos
Concatenado titulo com texto para melhoria do entendimento do sentimento e emoção

In [11]:
# Aplicado sob a coluna em pt-BR
df['sentiment'] = df['opinion'].apply(SentimentAnalysis)
display(df.head(5))

,score,title,opinion,opinion_en,title_en,sentiment
53289,5,Excelente fornecedor!,Entregue antes do prazo!,delivered ahead of schedule,Excellent supplier,neutral
68254,5,Sempre perfeita,O produto é de excelente qualidade. Entrega an...,product is of excellent quality delivery befor...,always perfect,positive
50553,1,Atraso na entrega,Comprei achando que iria receber em três dias ...,I bought thinking I would receive it in three ...,Delay in delivery,negative
84578,4,Bom,"Produto bom, pelo preço.",Good product for the price,good,positive
89538,2,Regular,Comprei 2 produtos chegou apenas 1. Aguardo re...,I bought 2 products arrived only 1 I wait for ...,regular,negative


In [23]:
df['emotion_text'] = df['title_en'] + " " + df['opinion_en']
df['emotion'] = df['emotion_text'].apply(EmotionAnalysis)
df = df.drop('emotion_text', axis='columns')

In [24]:
display(df)

,score,title,opinion,opinion_en,title_en,sentiment,emotion
53289,5,Excelente fornecedor!,Entregue antes do prazo!,delivered ahead of schedule,Excellent supplier,neutral,joy
68254,5,Sempre perfeita,O produto é de excelente qualidade. Entrega an...,product is of excellent quality delivery befor...,always perfect,positive,joy
50553,1,Atraso na entrega,Comprei achando que iria receber em três dias ...,I bought thinking I would receive it in three ...,Delay in delivery,negative,sadness
84578,4,Bom,"Produto bom, pelo preço.",Good product for the price,good,positive,joy
89538,2,Regular,Comprei 2 produtos chegou apenas 1. Aguardo re...,I bought 2 products arrived only 1 I wait for ...,regular,negative,joy
46800,5,super recomendado,muito bom,Very good,Super recommended,positive,joy
31170,5,Bom demais!,A entrega do produto veio antes do esperado. A...,delivery product came before expected quality ...,Too good,positive,joy
57893,5,Compra do produto Rebotec,Estou satisfeito com a entrega do produto. Par...,I am satisfied with delivery product congratul...,Buy Rebotec product,positive,joy
16727,5,Tudo ok,Otimo. Top de linha,Great top of the line,everything ok,positive,joy
44782,1,cancelamento compra,Foi cancelado a compra antes da emissão de not...,Purchase was canceled before the invoice was i...,Cancellation purchase,negative,sadness
